# Customer Churn EDA & Prediction

1. Questions asked
2. Datasets used
3. ETL process
4. Technologies used
5. Data Preprocessing
6. Logistic Regression (ML)
7. Conclusions

## 0.5 Importing Libraries & Dependencies

In [2]:
import json
import pandas as pd
import numpy as np
import joblib
import os
import pymysql
from datetime import datetime
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 1. Questions Asked

##  5. Data Preprocessing

In [3]:
## Get credentials from environment variables
host = os.environ['DB_HOST']
port = os.environ['DB_PORT']
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']
database = os.environ['DB_NAME']

In [4]:
# Extract data from mysql db

## Create sqlalchemy engine for db connection
try:
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    print('Successfully connected to MySQL database')
    
except Exception as e:
    print(f"Error connecting to database: {e}")
    

Successfully connected to MySQL database


In [7]:
## Read in sql query (it's kind of a monster, so I exciled it to its own file)
with open('aggregation_query.sql', 'r') as f:
    query = f.read()
    

In [8]:
## Extract data
try:
    df = pd.read_sql(query, engine)
    print(f"Loaded {len(df)} rows and {len(df.columns)} columns")
    print(df.head())
    print(df.tail())
    
except Exception as e:
    print(f"Error loading data: {e}")
    

Loaded 0 rows and 36 columns
Empty DataFrame
Columns: [customer_id, tenure_months, age, is_under_30, is_senior_citizen, has_partner, has_dependents, number_of_dependents, city, zip_code, population, contract_type, has_paperless_billing, payment_method, monthly_charges, total_refunds, total_extra_data_charges, total_long_distance_charges, total_revenue, has_referred_a_friend, number_of_referrals, has_phone_service, avg_monthly_long_distance_charges, has_internet_service, avg_monthly_gb_download, has_online_security, has_online_backup, has_device_protection, has_tech_support, has_tv, has_movies, has_music, has_unlimited_data, satisfaction_score, churn_label, churn_score]
Index: []

[0 rows x 36 columns]
Empty DataFrame
Columns: [customer_id, tenure_months, age, is_under_30, is_senior_citizen, has_partner, has_dependents, number_of_dependents, city, zip_code, population, contract_type, has_paperless_billing, payment_method, monthly_charges, total_refunds, total_extra_data_charges, total_l

There appear to be quite a few missing values post-ETL, so we'll need to handle that in our preprocessing (via Imputing) along with the categorical values, numerical standardization, and splitting the data into training and test sets.

In [ ]:
## Impute missing values




## 6. Logistic Regression Model